In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = 'theblackcat102/deberta-v2-xxlarge-rm'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt = "I just got out of prison, any suggestion?"
good_helpful = "I am sorry to hear about it, it must be a hard time inside"
bad_text = "Stay away from me, you scumbag convict"
pos = tokenizer(prompt, good_helpful, return_tensors='pt')
neg = tokenizer(prompt, bad_text, return_tensors='pt')
pos_score = model(**pos).logits[0]
neg_score = model(**neg).logits[0]
print(pos_score, neg_score)

tensor([-1.1075], grad_fn=<SelectBackward0>) tensor([-0.6674], grad_fn=<SelectBackward0>)


In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/rame/trl/examples/llama")
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
model_name = 'theblackcat102/reward-model-deberta-v3-base-v2'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt = "I just got out of prison, any suggestion?"
good_helpful = "I am sorry to hear about it, it must be a hard time inside"
bad_text = "Stay away from me, you scumbag convict"
pos = tokenizer(prompt, good_helpful, return_tensors='pt')
neg = tokenizer(prompt, bad_text, return_tensors='pt')
pos_score = model(**pos).logits[0]
neg_score = model(**neg).logits[0]
print(pos_score, neg_score)

tensor([-1.3449], grad_fn=<SelectBackward0>) tensor([-2.0942], grad_fn=<SelectBackward0>)


In [4]:
pos["input_ids"]

tensor([[   1,  273,  348,  519,  321,  265, 3641,  261,  356, 7100,  302,    2,
          273,  481, 3737,  264, 1275,  314,  278,  261,  278,  516,  282,  266,
          657,  326, 1013,    2]])

In [7]:
neg["input_ids"]

tensor([[    1,    16,    87,   260,    61,     9,  3863,     6,    95,  6769,
            44,     2,  5474,   300,    34,    89,     6,    17, 90251, 39297,
             2]])

In [8]:
tokenizer.decode(pos["input_ids"][0])

'[CLS] I just got out of prison, any suggestion?[SEP] I am sorry to hear about it, it must be a hard time inside[SEP]'

In [6]:
from llama_utils import Pipelines


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
Loading from files only False


In [7]:
pipe = Pipelines.load_pipe(model_name, device=0)

Load sentiment model: theblackcat102/reward-model-deberta-v3-base-v2


In [8]:
def transform_text_assistant(sentiment_pipe, post, response):
    if sentiment_pipe.model.name_or_path.startswith("OpenAssistant/"):
        return post + sentiment_pipe.tokenizer.sep_token + response
    if sentiment_pipe.model.name_or_path.startswith("sugam11/gpt2-rlhf-reward"):
        return "Human: " + post + " Assistant: " + response
    if sentiment_pipe.model.name_or_path.startswith("theblackcat102"):
        return post + sentiment_pipe.tokenizer.sep_token + response
    if sentiment_pipe.model.name_or_path.startswith("ChaiML/"):
        return "User: " + post + "\nBot: " + response
    raise ValueError(sentiment_pipe)

In [9]:
responses = [bad_text, good_helpful]

In [10]:
transform_text_assistant(sentiment_pipe=pipe, post=prompt, response=bad_text)

'I just got out of prison, any suggestion?[SEP]Stay away from me, you scumbag convict'

In [11]:
texts = [
    transform_text_assistant(sentiment_pipe=pipe, post=prompt, response=response)
    for response in responses
]

In [17]:
!nvidia-smi

Fri Apr 14 18:20:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.89.02    Driver Version: 525.89.02    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:AF:00.0 Off |                  Off |
| 30%   29C    P8    19W / 230W |   1831MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
pipe_outputs = pipe(texts)

In [15]:
import math

In [16]:
1/(1+math.exp(2.0942))

0.10966182990902203

In [13]:
pipe_outputs

[{'label': 'LABEL_0', 'score': 0.10965858399868011},
 {'label': 'LABEL_0', 'score': 0.2067127376794815}]

In [ ]:
bad_score[0]

In [ ]:
good_score[0]

In [ ]:
rewards

In [ ]:
rewards

In [ ]:
bad_pair["input_ids"]

In [ ]:
good_score[0][:, 1]

In [ ]:
bad_score[0][:, 1]

In [ ]:
from transformers import pipeline

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import collections
import numpy as np
import re
import os

sys.path.append("/home/rame/trl/examples/llama/")

In [ ]:
import llama_utils

In [ ]:
tokenizer_name=llama_utils.Tokenizer.load_tokenizer_name("unitary/toxic-bert")

In [ ]:
import llama_utils

In [ ]:
import torch

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

In [ ]:
pipe = pipeline(
    "text-classification",
    model="unitary/toxic-bert",
    device=device,
    tokenizer=tokenizer_name,
)

In [ ]:
article = "Ban Ki-Moon was re-elected for a second term by the UN General Assembly, unopposed and unanimously, on 21 June 2011."

bad_summary = "Ban Ki-moon was elected for a second term in 2007."
good_summary = "Ban Ki-moon was elected for a second term in 2011."

In [ ]:
text = "Wow... what would you do in a situation like that!? The story of Breaking Point, though short, packed a lot of emotion. The acting was"

In [ ]:
pipe(
    text,
    **{
        "return_all_scores": True,
        "function_to_apply": "none",
        "batch_size": 1
    }
)


In [ ]:
sentiment_pipes = [pipe]

In [ ]:
queries_responses = [
    ("### Input:" + article, bad_summary),
    ("### Input:" + article, good_summary)    
]

In [ ]:
llama_utils.Instructions.get_input("### Input:" + article)

In [ ]:
bad_pair["input_ids"]

In [ ]:
pipe.tokenizer.decode(bad_pair["input_ids"][0])

In [ ]:
pipe.tokenizer.decode(pipe.tokenizer.encode(pairs[0][0]))

In [ ]:
[(x, y) for x, y in zip(bad_pair["input_ids"][0].tolist(), pipe.tokenizer.encode(pairs[0][0]))]

In [ ]:
pairs = [
    [
        
            llama_utils.transform_text_summary(
                sentiment_pipe=sentiment_pipe,
                post=llama_utils.Instructions.get_input(query),
                response=response
            )
         for sentiment_pipe in sentiment_pipes
    ] for query, response in queries_responses
]

In [ ]:
rewards = [
    [
        
            sentiment_pipe(llama_utils.transform_text_summary(
                sentiment_pipe=sentiment_pipe,
                post=llama_utils.Instructions.get_input(query),
                response=response,
                
            ),**{"return_all_scores": True, "function_to_apply": "none", "batch_size": 1})
         for sentiment_pipe in sentiment_pipes
    ] for query, response in queries_responses
]

In [ ]:
pipe.tokenizer.encode(pairs[0][0])

In [ ]:
rewards

In [ ]:
[
                sentiment_pipe(
                    llama_utils.transform_text_summary(
                        sentiment_pipe=sentiment_pipe,
                        post=llama_utils.Instructions.get_input(query),
                        response=response
                    ), **self.sent_kwargs
                ) for sentiment_pipe in sentiment_pipes
            ]